# <center>Подготовка данных (индекс S&P100)
Подготовим данные для задачи.   
Данные будут представлять из себя многомерный временной ряд из курсов акций, входящих в индекс (101 акция) плюс сам индекс. Итого **102-мерный** вектор. Здесь и далее работать будем с индексом **S&P100** (тикер **OEX**). Для прогнозирования везде будем брать цену закрытия (**"Close"**)

In [1]:
import os
os.chdir("..")

%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import pandas as pd
import json

In [3]:
path_to_prices_raw = 'prices_sp100_15min_raw'
path_to_prices = 'prices_sp100_15min'
path_to_data = 'data'

список акций индекса (версия _**corr**_)

In [4]:
with open("sp100_tickers_corr.json", "r") as f:
    sp100_tickers = json.load(f)
print(sp100_tickers)
len(sp100_tickers)

['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'AIG', 'ALL', 'AMGN', 'AMT', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BKNG', 'BLK', 'BMY', 'BRK-B', 'C', 'CAT', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CRM', 'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FB', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD', 'MDLZ', 'MDT', 'MET', 'MMM', 'MO', 'MRK', 'MS', 'MSFT', 'NEE', 'NFLX', 'NKE', 'NVDA', 'ORCL', 'OXY', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTX', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TMO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'V', 'VZ', 'WBA', 'WFC', 'WMT', 'XOM']


101

In [8]:
def data_to_one_file(ticker):
    years = 2
    months = 12
    data = pd.DataFrame()
    for year in range(1, years+1):
        for month in range(1, months+1):
            file_path = os.path.join(path_to_prices_raw, f"{ticker}_y{year}_m{month}.csv")
            if os.path.exists(file_path):
                df = pd.read_csv(file_path, index_col=0)
                data = pd.concat([data, df], axis=0)
    data.sort_index(inplace=True)
    return data

#### тесты

In [9]:
ticker = "IBM"
data = data_to_one_file(ticker)

data.head()

,open,high,low,close,volume
time,,,,,
2018-10-15 05:00:00,127.413221,127.413221,127.222496,127.222496,1083
2018-10-15 07:45:00,127.594864,127.603946,127.594864,127.603946,599
2018-10-15 08:00:00,127.603946,127.603946,127.603946,127.603946,150
2018-10-15 08:15:00,127.912738,127.912738,127.812835,127.812835,934
2018-10-15 08:30:00,128.094290,128.112545,128.012551,128.012551,515


In [10]:
data.shape

(20225, 5)

In [11]:
data.tail()

,open,high,low,close,volume
time,,,,,
2020-10-01 17:45:00,121.09,121.09,121.09,121.09,449
2020-10-01 18:15:00,121.14,121.14,121.14,121.14,100
2020-10-01 18:45:00,121.00,121.00,121.00,121.00,537
2020-10-01 19:45:00,121.10,121.10,121.09,121.10,1362
2020-10-01 20:00:00,121.10,121.10,121.10,121.10,300


In [12]:
data.to_csv(os.path.join(path_to_prices, f"{ticker}.csv"))

In [13]:
data_ = pd.read_csv(os.path.join(path_to_prices, f"{ticker}.csv"), index_col=0)
data_.head()

,open,high,low,close,volume
time,,,,,
2018-10-15 05:00:00,127.413221,127.413221,127.222496,127.222496,1083
2018-10-15 07:45:00,127.594864,127.603946,127.594864,127.603946,599
2018-10-15 08:00:00,127.603946,127.603946,127.603946,127.603946,150
2018-10-15 08:15:00,127.912738,127.912738,127.812835,127.812835,934
2018-10-15 08:30:00,128.094290,128.112545,128.012551,128.012551,515


#### формируем файлы по тикерам

In [16]:
end_num = None

In [17]:
%%time
for ticker in sp100_tickers[:end_num]:
    data = data_to_one_file(ticker)
    data.to_csv(os.path.join(path_to_prices, f"{ticker}.csv"))

Wall time: 42.8 s


#### формируем рабочий файл с котировками

In [19]:
%%time
data = pd.DataFrame()
for ticker in sp100_tickers[:end_num]:
    df = pd.read_csv(os.path.join(path_to_prices, f"{ticker}.csv"), index_col=0)
    df.rename({"close": ticker}, axis=1, inplace=True)
    data = pd.concat([data, pd.DataFrame(df[ticker])], axis=1, join='outer', sort=True)
data.head()

Wall time: 7.98 s


,AAPL,ABBV,ABT,ACN,ADBE,AIG,ALL,AMGN,AMT,AMZN,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
2018-10-15 04:15:00,53.728530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1776.49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-15 04:30:00,53.550596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1772.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-15 04:45:00,53.587158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1772.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-15 05:00:00,53.453099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-15 05:15:00,NaN,NaN,NaN,NaN,245.01,NaN,NaN,NaN,NaN,1773.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
data.shape

(31627, 101)

In [22]:
data.tail(100)

,AAPL,ABBV,ABT,ACN,ADBE,AIG,ALL,AMGN,AMT,AMZN,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
2020-10-01 11:15:00,116.3923,87.7400,109.5400,226.5700,496.8500,27.5950,92.940,254.7100,243.4175,3185.3899,...,311.5950,195.9700,169.7000,35.8400,204.4400,59.3300,35.7800,23.3200,143.2001,33.2500
2020-10-01 11:30:00,116.6500,87.6425,109.5000,226.6700,498.3500,27.6550,93.310,255.2400,243.6800,3197.1891,...,311.7000,195.9700,169.3300,35.9400,205.0450,59.4700,35.8650,23.2800,142.9700,33.2750
2020-10-01 11:45:00,116.6728,87.4100,109.6000,226.5600,499.0300,27.6200,93.310,255.8900,243.7850,3200.9200,...,311.9400,195.6500,169.3700,35.8600,204.8300,59.4075,35.9600,23.2650,143.0650,33.2700
2020-10-01 12:00:00,116.3150,87.4100,109.7700,226.6100,497.8250,27.7250,93.480,256.4300,244.7600,3195.6623,...,312.4800,196.2100,169.8400,35.8100,204.1750,59.4000,35.8800,23.2550,142.9760,33.2850
2020-10-01 12:15:00,116.1200,87.2700,109.5800,226.1200,497.7971,27.6275,93.080,256.3950,243.9500,3193.3350,...,312.4400,195.7800,169.8100,35.8250,203.7350,59.4250,35.9050,23.2599,143.2700,33.2286
2020-10-01 12:30:00,116.2508,87.4650,109.5500,226.2900,497.2400,27.6450,93.290,256.7900,244.1000,3190.1600,...,312.9900,196.3000,170.7500,35.9300,204.0300,59.5700,35.9250,23.3400,143.9300,33.2150
2020-10-01 12:45:00,116.1799,87.6781,109.5700,226.2400,496.5100,27.7350,93.430,257.0424,243.8600,3190.0500,...,314.1000,196.8500,170.7100,35.9800,203.9000,59.5550,35.9950,23.4300,144.1400,33.2200
2020-10-01 13:00:00,116.3300,87.4600,109.5700,226.1750,497.0450,27.6650,93.290,256.5950,242.7850,3202.3000,...,313.3000,196.2800,170.3950,35.9611,203.9500,59.4550,35.9400,23.4200,143.8600,33.1416
2020-10-01 13:15:00,116.4400,87.5400,109.4600,226.0600,498.2700,27.6950,92.920,256.3350,243.3100,3203.3900,...,312.4700,195.5650,169.8897,36.0100,203.9900,59.4800,35.9602,23.4200,143.9398,33.1480
2020-10-01 13:30:00,116.3815,87.4100,109.2700,225.7300,498.5700,27.6900,93.110,256.0500,243.4343,3204.2900,...,312.8000,194.6000,169.0050,35.9200,203.5800,59.4600,35.8950,23.4000,143.3000,33.1600


In [24]:
data.loc[:"2020-10-01 17:00:00"].tail()

,AAPL,ABBV,ABT,ACN,ADBE,AIG,ALL,AMGN,AMT,AMZN,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
2020-10-01 16:00:00,116.8000,87.1500,108.6400,225.3500,499.52,27.72,92.26,255.5000,241.98,3222.01,...,313.0700,194.30,167.65,36.0,203.3400,59.47,35.870,23.5600,143.0700,33.13
2020-10-01 16:15:00,116.7800,87.1487,108.6509,225.2025,499.46,27.72,92.26,255.3645,242.15,3221.00,...,313.0387,194.23,167.65,36.0,203.3297,59.47,36.090,23.5100,143.0657,33.15
2020-10-01 16:30:00,116.7601,87.2000,NaN,225.1800,499.51,NaN,NaN,NaN,242.15,3219.07,...,NaN,NaN,NaN,36.0,203.3500,NaN,NaN,23.4603,143.1500,33.12
2020-10-01 16:45:00,116.8200,87.3100,NaN,NaN,499.85,NaN,NaN,NaN,NaN,3220.90,...,313.0700,194.23,NaN,36.0,203.1700,NaN,35.859,23.5199,143.0200,33.14
2020-10-01 17:00:00,116.7700,87.2500,108.5800,NaN,500.20,NaN,NaN,NaN,NaN,3219.20,...,NaN,NaN,NaN,36.0,NaN,NaN,NaN,23.4801,143.0000,33.14


подчистим немного данные

In [25]:
data_ = data.loc[:"2020-10-01 17:00:00"].copy()

Видим, что у части акций нет дальних котировок (по-видимому, они начали торговаться позже). Заменим пропуски предыдущими  котировками (с этим тоже можно поработать, но пока так).

In [26]:
data_filled = data_.fillna(method='ffill')
data_filled.head()

,AAPL,ABBV,ABT,ACN,ADBE,AIG,ALL,AMGN,AMT,AMZN,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
2018-10-15 04:15:00,53.728530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1776.49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-15 04:30:00,53.550596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1772.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-15 04:45:00,53.587158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1772.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-15 05:00:00,53.453099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1772.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-15 05:15:00,53.453099,NaN,NaN,NaN,245.01,NaN,NaN,NaN,NaN,1773.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
data_filled.tail()

,AAPL,ABBV,ABT,ACN,ADBE,AIG,ALL,AMGN,AMT,AMZN,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
2020-10-01 16:00:00,116.8000,87.1500,108.6400,225.3500,499.52,27.72,92.26,255.5000,241.98,3222.01,...,313.0700,194.30,167.65,36.0,203.3400,59.47,35.870,23.5600,143.0700,33.13
2020-10-01 16:15:00,116.7800,87.1487,108.6509,225.2025,499.46,27.72,92.26,255.3645,242.15,3221.00,...,313.0387,194.23,167.65,36.0,203.3297,59.47,36.090,23.5100,143.0657,33.15
2020-10-01 16:30:00,116.7601,87.2000,108.6509,225.1800,499.51,27.72,92.26,255.3645,242.15,3219.07,...,313.0387,194.23,167.65,36.0,203.3500,59.47,36.090,23.4603,143.1500,33.12
2020-10-01 16:45:00,116.8200,87.3100,108.6509,225.1800,499.85,27.72,92.26,255.3645,242.15,3220.90,...,313.0700,194.23,167.65,36.0,203.1700,59.47,35.859,23.5199,143.0200,33.14
2020-10-01 17:00:00,116.7700,87.2500,108.5800,225.1800,500.20,27.72,92.26,255.3645,242.15,3219.20,...,313.0700,194.23,167.65,36.0,203.1700,59.47,35.859,23.4801,143.0000,33.14


In [28]:
data_filled.shape

(31551, 101)

In [29]:
data_filled.dropna().shape

(8032, 101)

чтобы не терять данные, сделаем такое заполнение пропусков

In [31]:
data_filled = data_filled.fillna(method='backfill')
data_filled.dropna().shape

(31551, 101)

Сохраним этот файл и будем дальше работать уже с ним

In [32]:
filename = "sp100_15min.csv"
data_filled.dropna().to_csv(os.path.join(path_to_data, filename))

#### тестим сохранение-загрузку

In [33]:
data_loaded = pd.read_csv(os.path.join(path_to_data, filename), index_col=0)
data_loaded.head()

,AAPL,ABBV,ABT,ACN,ADBE,AIG,ALL,AMGN,AMT,AMZN,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
2018-10-15 04:15:00,53.728530,82.335268,67.871121,153.710081,245.01,46.671585,89.674787,186.465966,136.580973,1776.49,...,254.662905,145.711303,108.035108,47.173193,136.250155,49.771425,69.860241,49.019968,90.07355,72.569641
2018-10-15 04:30:00,53.550596,82.335268,67.871121,153.710081,245.01,46.671585,89.674787,186.465966,136.580973,1772.50,...,254.662905,145.711303,108.035108,47.173193,136.250155,49.771425,69.860241,49.019968,90.07355,72.569641
2018-10-15 04:45:00,53.587158,82.335268,67.871121,153.710081,245.01,46.671585,89.674787,186.465966,136.580973,1772.00,...,254.662905,145.711303,108.035108,47.173193,136.250155,49.771425,69.860241,49.019968,90.07355,72.569641
2018-10-15 05:00:00,53.453099,82.335268,67.871121,153.710081,245.01,46.671585,89.674787,186.465966,136.580973,1772.00,...,254.662905,145.711303,108.035108,47.173193,136.250155,49.771425,69.860241,49.019968,90.07355,72.569641
2018-10-15 05:15:00,53.453099,82.335268,67.871121,153.710081,245.01,46.671585,89.674787,186.465966,136.580973,1773.37,...,254.662905,145.711303,108.035108,47.173193,136.250155,49.771425,69.860241,49.019968,90.07355,72.569641


In [34]:
data_loaded.shape

(31551, 101)

In [35]:
data_loaded.tail()

,AAPL,ABBV,ABT,ACN,ADBE,AIG,ALL,AMGN,AMT,AMZN,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
2020-10-01 16:00:00,116.8000,87.1500,108.6400,225.3500,499.52,27.72,92.26,255.5000,241.98,3222.01,...,313.0700,194.30,167.65,36.0,203.3400,59.47,35.870,23.5600,143.0700,33.13
2020-10-01 16:15:00,116.7800,87.1487,108.6509,225.2025,499.46,27.72,92.26,255.3645,242.15,3221.00,...,313.0387,194.23,167.65,36.0,203.3297,59.47,36.090,23.5100,143.0657,33.15
2020-10-01 16:30:00,116.7601,87.2000,108.6509,225.1800,499.51,27.72,92.26,255.3645,242.15,3219.07,...,313.0387,194.23,167.65,36.0,203.3500,59.47,36.090,23.4603,143.1500,33.12
2020-10-01 16:45:00,116.8200,87.3100,108.6509,225.1800,499.85,27.72,92.26,255.3645,242.15,3220.90,...,313.0700,194.23,167.65,36.0,203.1700,59.47,35.859,23.5199,143.0200,33.14
2020-10-01 17:00:00,116.7700,87.2500,108.5800,225.1800,500.20,27.72,92.26,255.3645,242.15,3219.20,...,313.0700,194.23,167.65,36.0,203.1700,59.47,35.859,23.4801,143.0000,33.14


In [36]:
data_loaded.shape

(31551, 101)